# <font color=darkolivegreen> Анализ поведения пользователей мобильного приложения по продаже продуктов питания </font> 

## <font color=olive>Описание данных</font>

#### <font color=olivedrab>Таблица logs_exp:</font>
 Каждая запись в логе — это действие пользователя, или событие.
 - EventName — название события;
 - DeviceIDHash **— уникальный идентификатор пользователя;
 - EventTimestamp — время события;
 - ExpId — номер эксперимента: 246 и 247 — контрольные группы, а 248 — экспериментальная.

#### <font color=olivedrab>Оглавление </font>

1. [Открытие данных](#открытие)
2. [Предобработка данных](#предобработка)
    * [Замена названий столбцов](#замена_названий_столбцов)
    * [Замена названий событий](#замена_названий_событий)
    * [Проверка и обработка дубликатов](#проверка_и_обработка_дубликатов)
    * [Обработка даты](#обработка_даты)
    * [Вывод](#вывод_предобработка)  
3. [Изучение данных](#изучение_данных)
    * [Количество событий и пользователей](#количество_событий_и_пользователей)
    * [Количество событий по датам](#количество_событий_по_датам)
    * [Вывод](#вывод_изучение)
4. [Воронка событий](#воронка)
    * [Количество событий по видам](#количество_событий_по_видам)
    * [Воронка событий](#воронка_событий_без_учета_последовательности)    
    * [Воронка событий с учетом последовательности событий](#построение_воронки_с_учетом_последовательности_событий)    
    * [Вывод](#вывод_воронка)
5. [Результаты эксперимента](#эксперимент)
    * [Сравнение контрольных групп](#сравнение_контрольных_групп)
    * [Сравнение первой контрольной группы с экспериментальной](#сравнение_контрольной_группы_246_с_экспериментальной)  
    * [Сравнение второй контрольной группы с экспериментальной](#сравнение_контрольной_группы_247_с_экспериментальной)  
    * [Сравнение контрольной группы с экспериментальной](#сравнение_объединенной_контрольной_группы_с_экспериментальной)
    * [Вывод](#вывод_эксперимент)
6. [Общий вывод](#вывод)

### <font color=olive>Шаг 1. Загрузка данных</font>

In [54]:
# импортируем данные
import pandas as pd
from datetime import datetime, time
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats as st
import seaborn as sns
from matplotlib import pyplot as plt
from plotly import graph_objects as go
import plotly.express as px
import math as mth

<a id="открытие"></a>

In [55]:
# и сохраним исходную таблицу в переменную:
logs = pd.read_csv('/datasets/logs_exp.csv', sep='\t')
logs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244126 entries, 0 to 244125
Data columns (total 4 columns):
EventName         244126 non-null object
DeviceIDHash      244126 non-null int64
EventTimestamp    244126 non-null int64
ExpId             244126 non-null int64
dtypes: int64(3), object(1)
memory usage: 7.5+ MB


In [56]:
logs['EventName'].value_counts()

MainScreenAppear           119205
OffersScreenAppear          46825
CartScreenAppear            42731
PaymentScreenSuccessful     34313
Tutorial                     1052
Name: EventName, dtype: int64

В таблице представлены данные о 244 125 событиях:
 - MainScreenAppear - переход на главный экран приложения;
 - OffersScreenAppear - переход на страницу заказа товара;
 - CartScreenAppear - добавление товара в корзину;
 - PaymentScreenSuccessful - переход на страницу оплаты;
 - Tutorial - переход на страницу обучения. Строк с этим событием в общей количестве очень мало
 
Для каждого события указаны
 - уникальный идентификатор пользователя;
 - время совершения события;
 - номер эксперимента (для отнесения к одной из групп А/А/B-теста. 246 и 247 — контрольные группы, а 248 — экспериментальная.
 
 Все строки заполнены, тип данных столбца с временем события необходимо привести к дате.

### <font color=olive>Шаг 2. Подготовьте данные</font> <a id="предобработка"></a>

<a id="замена_названий_столбцов"></a>

In [57]:
# заменим названия столбцов на удобные:
logs = logs.rename(columns={'EventName': 'event', 'DeviceIDHash': 'user_id', 'EventTimestamp': 'timestamp_unix', 'ExpId': 'exp_id'})
logs.head(5)

,event,user_id,timestamp_unix,exp_id
0,MainScreenAppear,4575588528974610257,1564029816,246
1,MainScreenAppear,7416695313311560658,1564053102,246
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248


<a id="замена_названий_событий"></a>

In [58]:
# заменим названия события из столбца event на удобные:
def events(row):
    
    event = row['event']
  
    if event == "MainScreenAppear":
        return 'main'
   
    if event == "OffersScreenAppear":
        return 'offer'
    
    if event == "CartScreenAppear":
        return 'cart'
    
    if event == "PaymentScreenSuccessful":
        return 'payment'
    
    if event == "Tutorial":
        return 'tutorial'
# и добавим колонку в таблицу    
logs['event'] = logs.apply(events, axis=1)

<a id="проверка_и_обработка_дубликатов"></a>

In [59]:
# проверим на наличие дубликатов строк
logs.duplicated().sum()

413

В таблице содержится 413 дублирующих строк. Удалим их.

In [60]:
logs = logs.drop_duplicates()

<a id="обработка_даты"></a>

In [61]:
# добавим столбец timestamp с информацией о дате и времени события, изменив тип данных столбца timestamp_unix на дату:
logs['timestamp'] = pd.to_datetime(logs['timestamp_unix'], unit = 's')

In [62]:
# выделим из полученного столбца дату события:
#logs['date'] = pd.DatetimeIndex(logs['timestamp']).date
logs['date'] = logs['timestamp'].dt.date
logs['date'] = logs['date'].astype('datetime64[ns]')

### <font color=olive>Вывод</font> <a id="вывод_предобработка"></a>

Данные были подготовлены для дальнейшего анализа:
 - заменены названия столбцов на более удобные;
 - заменены названия событий из столбца event на более простые и понятные, но не меняющие первоначальный смысл;
 - удалены дубликаты строк, после чего из 244 125 осталось 243 713 строк;
 - добавлен столбец с датой и временем события 'timestamp' в стандартном формате год-месяц-дата час-минута-секунда;
 - добавлен столбец только с датой события в формате год-месяц-дата.
 
Данные готовы для дальнейшего анализа.

### <font color=olive>Шаг 3. Изучите и проверьте данные</font> <a id="изучение_данных"></a> 

<a id="количество_событий_и_пользователей"></a>

In [63]:
# посчитаем сколько всего событий в логе:
len(logs)

243713

После предобработки и удаления дубликатов в таблице представлены данные о 243 713 событиях.

In [64]:
# посчитаем сколько всего пользователей в логе:
logs['user_id'].nunique()

7551

В логе представлена информация о действиях 7 551 пользователя.

In [65]:
# посчитаем сколько в среднем событий приходится на пользователя за весь период лога:

len(logs) / logs['user_id'].nunique()

32.27559263673685

За период лога на 1 пользователя в среднем приходится 32 события.

<a id="количество_событий_по_датам"></a>

In [66]:
logs.groupby('date')['event'].count().reset_index()

,date,event
0,2019-07-25,9
1,2019-07-26,31
2,2019-07-27,55
3,2019-07-28,105
4,2019-07-29,184
5,2019-07-30,412
6,2019-07-31,2030
7,2019-08-01,36141
8,2019-08-02,35554
9,2019-08-03,33282


In [67]:
fig_one = px.bar(logs.groupby('date')['event'].count().reset_index(), x='date', y='event', title='Количество событий по датам', color = 'event')

fig_one.update_xaxes(tickangle=45)
fig_one.update_layout(xaxis_title="Дата",
                  yaxis_title="Количество событий, шт.",)

fig_one.show()

В логах представлены данные с 25 июля по 7 августа. С 25 июля по 31 июля включительно очень мало данных за каждый день. Скорее всего это связано с тем, что в логи новых дней по некоторым пользователям могут «доезжать» события из прошлого — это может «перекашивать данные». Не будем учитывать данные за этот период в дальнейшем анализе:

In [68]:
logs = logs[logs['date'] > '2019-07-31']
logs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240887 entries, 2828 to 244125
Data columns (total 6 columns):
event             240887 non-null object
user_id           240887 non-null int64
timestamp_unix    240887 non-null int64
exp_id            240887 non-null int64
timestamp         240887 non-null datetime64[ns]
date              240887 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(3), object(1)
memory usage: 12.9+ MB


После фильтрации данных по дате в таблице осталось 240 887 строк. От исходной таблицы было отрезано не более 1,5% строк. Посчитаем сколько в среднем событий приходится на пользователя по отфильтрованным данным:

In [69]:
len(logs) / logs['user_id'].nunique()

31.97332094504911

Это значение сильно не изменилось. Проверим количество и наличие пользователей в группах A/A/B-теста:

In [70]:
logs.groupby('exp_id')['user_id'].nunique()

exp_id
246    2484
247    2513
248    2537
Name: user_id, dtype: int64

Количество уникальных пользователей для каждой из групп примерно равны между собой.

### <font color=olive>Вывод</font> <a id="вывод_изучение"></a>

В исходной таблице были представлены данные о событиях в мобильном приложении с 25 июля по 07 августа 2019 года. При детальном рассмотрении было обнаружено, что в период с 25 по 31 июля лог содержит очень мало информации за каждую из дат, а с 01 августа, судя по содержаню событий, лог можно считать полным. Для дальнейшего анализа было исключено менее 1,5% событий. Это исключение не может сильно повлиять на конечный результат.

### <font color=olive>Шаг 4. Изучите воронку событий</font> <a id="воронка"></a>

<a id="количество_событий_по_видам"></a>

In [71]:
# рассмотрим события, которые есть в логах и как часто они встречаются:
events = logs['event'].value_counts().reset_index().rename(columns={'index': 'event', 'event': 'count'})
total = events['count'][0]
events['ratio'] = (events['count'] * 100 / events['count'].sum()).round()
events

,event,count,ratio
0,main,117328,49.0
1,offer,46333,19.0
2,cart,42303,18.0
3,payment,33918,14.0
4,tutorial,1005,0.0


In [72]:
fig_two = px.bar(events, x='event', y='count', title='Количество событий по видам')

fig_two.update_xaxes(tickangle=45)
fig_two.update_layout(xaxis_title="Событие",
                  yaxis_title="Количество событий, шт.",)

fig_two.show()

Наиболее часто происходящее событие - это переход на главную страницу приложения, то есть вход в него, этот тип события составлет практически полоивину всех действий за исследумый период, а именно 49%. Переход на страницу заказа товаров составляет 19% от всего количества событий, немного меньше - 18% - составляет событие добавление товара в корзину. Оплату товара произвели в 14% случаев. Так же в таблице содержится информация о 1 005 случаях прохождения туториала, но это количество настолько маленькое, что составляет менее 1% всех событий.

<a id="воронка_событий_без_учета_последовательности"></a>

In [73]:
# построим воронку событий:
funnel = (logs.groupby('event')['user_id'].nunique()
                                 .reset_index()
                                 .rename(columns={'user_id': 'count'})
                                 .sort_values(by = 'count', ascending = False)
                                 .set_index('event')
                                 .reset_index())

funnel['share'] = (funnel['count'] * 100 / logs['user_id'].nunique()).round()
funnel

,event,count,share
0,main,7419,98.0
1,offer,4593,61.0
2,cart,3734,50.0
3,payment,3539,47.0
4,tutorial,840,11.0


В нашем случае 7 419 пользователей зашли на главную страницу приложения. Это количество составляет 98% от общего количества уникальных пользователей, по которым есть информация в таблице. Значит 2% пользователей каким-то образом избежали посещения главного экрана приложения и выполнили последующие шаги. 4 593 пользователя перешли на страницу заказа товара (что составляет 61% от общего количества пользователей), затем 3 734 пользователя добавили товар в корзину (50%), 3 539 пользователей (47%) оплатили заказ. Так же в таблице содержатся сведения о 840 пользователях, которые прошли туториал, который показывается пользователям при первом запуске приложения, но скорее всего его можно пройти в любой момент пользования приложением, поэтому его нельзя вписать в какой-то определенный этап продуктовой воронки. Не будем учитывать события этого типа при дальнейшем анализе:

In [74]:
logs = logs.query('event not in ("tutorial")')

<a id="построение_воронки_с_учетом_последовательности_событий"></a>

На предыдущем шаге обнаружилось, что только 98% пользователей были на главной странице приложения. Дальнейший анализ будем проводить исходя из такой последовательности событий:
    1. переход на главную;
    2. переход на страницу заказа товара;
    3. добавление товара в корзину;
    4. оплата заказа.
    
Для фильтрации таблицы с учетом последовательности событий нужно сначала по каждому пользователю посчитать, в какое время он впервые совершил определённое событие. Затем найти тех, чьи события происходили в нужном порядке. 

In [110]:
# считаем время первого срабатывания каждого события методом для каждого пользователя:
users = logs.pivot_table(
        index='user_id', 
        columns='event', 
        values='timestamp',
        aggfunc='min').reset_index()
users = users[['user_id', 'main', 'offer', 'cart', 'payment']].set_index('user_id')
users
# посчитаем количество зашедших в приложение пользователей, для этого найдём число строк таблицы users,
# где в столбце main не пустое значение:
step_1 = ~users['main'].isna()

# посчитаем количество пользователей из тех, кто заходил в приложение, которые перешли на страницу заказа товара, то есть тех,
# у кого, во-первых, есть дата посещения сайта, а во-вторых, дата события «offer» позже даты первого захода на сайт:
step_2 = step_1 & (users['offer'] >= users['main'])

# посчитаем количество пользователей из предыдущей группы, но дата события "cart" позже даты события "offer":
step_3 = step_2 & (users['cart'] >= users['offer'])

# аналогично для события payment:
step_4 = step_3 & (users['payment'] >= users['cart'])

# посчитаем количество пользователей для каждого из событий
n_main = users[step_1].shape[0]
n_offer = users[step_2].shape[0]
n_cart = users[step_3].shape[0]
n_payment = users[step_4].shape[0]

# создадим финальную продуктовую воронку, в которой будет указано количество пользователей для каждого из событий
# и доля пользователей проходящих на следующи шаг воронки, рассчитанная как отношение количества пользователей на текущем шаге
# к количеству пользователей на предыдущем:
finish_funnel = pd.DataFrame({"event": pd.Series(["main", "offer", "cart", "payment"]),
                              "count": pd.Series([n_main, n_offer, n_cart, n_payment]),
                              "share": pd.Series([100, (n_offer * 100 / n_main), (n_cart * 100 / n_offer),\
                                                  (n_payment * 100 / n_cart)])})

finish_funnel['share'] = finish_funnel['share'].astype('int64').round()

# добавим в финальную продуктовую воронку столбец с первоначальным количеством пользователей на каждом этапе пользования
# приложением 
finish_funnel['count_first'] = funnel.query('event != "tutorial"')['count']

In [76]:
# построим воронку продаж для случаев без учета последовательности событий и с учетом последовательности событий:
fig_three = go.Figure()


fig_three.add_trace(go.Funnelarea(
    scalegroup = "second", values = finish_funnel['count'], textinfo = "value", labels = finish_funnel['event'],
    title = {"position": "top center", "text": "Воронка продаж с учетом последовательности событий"},
    domain = {"x": [0, 0.5], "y": [0, 0.5]}))
    
    
    
fig_three.add_trace(go.Funnelarea(
    scalegroup = "first", values = finish_funnel['count_first'], labels = finish_funnel['event'],
    textinfo = "value",
    title = {"position": "top center", "text": "Воронка продаж без учета последовательности событий"},
    domain = {"x": [0, 0.5], "y": [0.55, 1]}))



fig_three.update_layout(
            margin = {"l": 100, "r": 200}, shapes = [
            {"x0": 0, "x1": 0.5, "y0": 0, "y1": 0.5},
            {"x0": 0, "x1": 0.5, "y0": 0.55, "y1": 1}])
          
fig_three.show()

Из графиков видно, что воронки продаж без учета последовательности событий и с ней отличаются. В воронке без учета последовательности событий на шаг просмотра корзины переходит значительно больше пользователей, чем в воронке с учетом. Аналогично происходит и с шагом успешной оплаты заказа. Можно сделать вывод, что начать просматривать корзину с заказом, а затем и оплатить его можно минуя главный экран и страницу с предложением товаров.

In [77]:
finish_funnel[['event', 'count']]

,event,count
0,main,7419
1,offer,4202
2,cart,1796
3,payment,1360


In [78]:
# рассчитаем долю пользователей, которые доходят от первого события до оплаты:
((finish_funnel['count'][3] / finish_funnel['count'][0]) * 100).round(2)

18.33

Итоговая конверсия мобильного приложения с учетом последовательности событий составила 18,33%

### <font color=olive>Вывод</font> <a id="вывод_воронка"></a>

Наиболее часто происходящее событие в приложении - это переход на главную страницу, то есть вход в него, этот тип события составлет практически полоивину всех действий за исследумый период, а именно 49%. Переход на страницу заказа товаров составляет 19% от всего количества событий, немного меньше - 18% - составляет событие добавление товара в корзину. Оплату товара произвели в 14% случаев. Так же в таблице содержится информация о 1 005 случаях прохождения туториала, но это количество настолько маленькое, что составляет менее 1% всех событий.

В нашем случае 7 419 пользователей зашли на главную страницу приложения. Это количество составляет 98% от общего количества уникальных пользователей, по которым есть информация в таблице. Значит 2% пользователей каким-то образом избежали посещения главного экрана приложения и выполнили последующие шаги. 4 593 пользователя перешли на страницу заказа товара (что составляет 61% от общего количества пользователей), затем 3 734 пользователя добавили товар в корзину (50%), 3 539 пользователей (47%) оплатили заказ. Так же в таблице содержатся сведения о 840 пользователях, которые прошли туториал, который показывается пользователям при первом запуске приложения, но скорее всего его можно пройти в любой момент пользования приложением, поэтому его нельзя вписать в какой-то определенный этап продуктовой воронки. 

Обнаружилось, что только 98% пользователей были на главной странице приложения. Дальнейший анализ будем проводить исходя из такой последовательности событий:
    1. переход на главную;
    2. переход на страницу заказа товара;
    3. добавление товара в корзину;
    4. оплата заказа.

In [79]:
fig_three.show()

Из графиков видно, что воронки продаж без учета последовательности событий и с ней отличаются. В воронке без учета последовательности событий на шаг просмотра корзины переходит значительно больше пользователей, чем в воронке с учетом. Аналогично происходит и с шагом успешной оплаты заказа. Скорее всего это происходит потому, что у пользователей путь от прихода в приложение до покупки дольше, чем тот период, который мы выбрали. 

Итоговая конверсия мобильного приложения с учетом последовательности событий составила 18,33%.

### <font color=olive>Шаг 5. Изучите результаты эксперимента</font> <a id="эксперимент"></a>

In [80]:
logs.groupby('exp_id')['user_id'].nunique()

exp_id
246    2483
247    2512
248    2535
Name: user_id, dtype: int64

В контрольных группах А/А/В-теста 246 и 247 по 2 483 и 2512 пользователей соответственно, в экспериментальной группе для 2 535 пользователей изменили шрифт в приложении.

In [81]:
# считаем количество пользователей, совершивших каждое из событий в группе 246:
group_a1 = (logs.query('exp_id == 246').groupby('event')['user_id'].nunique()
                                                       .reset_index()
                                                       .rename(columns={'user_id': 'count_a1'})
                                                       .sort_values(by = 'count_a1', ascending = False)
                                                       .set_index('event')
                                                       .reset_index())

# считаем количество пользователей, совершивших каждое из событий в группе 247:
group_a2 = (logs.query('exp_id == 247').groupby('event')['user_id'].nunique()
                                                       .reset_index()
                                                       .rename(columns={'user_id': 'count_a2'})
                                                       .sort_values(by = 'count_a2', ascending = False)
                                                       .set_index('event')
                                                       .reset_index())

# считаем количество пользователей, совершивших каждое из событий в группе 248:
group_b = (logs.query('exp_id == 248').groupby('event')['user_id'].nunique()
                                                       .reset_index()
                                                       .rename(columns={'user_id': 'count_b'})
                                                       .sort_values(by = 'count_b', ascending = False)
                                                       .set_index('event')
                                                       .reset_index())

In [82]:
# объединяем посчитанных пользователей по группам:
merged_a1_a2 = group_a1.merge(group_a2)

# считаем количество пользователей, совершивших каждое из событий в объединенной группе 246+247:
merged_a1_a2['count_a'] = merged_a1_a2['count_a1'] + merged_a1_a2['count_a2']

events_by_group = merged_a1_a2.merge(group_b)

In [83]:
# количество уникальных пользователей в каждой из групп:
total_users_a1 = logs.query('exp_id == 246')['user_id'].nunique()
total_users_a2 = logs.query('exp_id == 247')['user_id'].nunique()
total_users_b = logs.query('exp_id == 248')['user_id'].nunique()

##### Проверим точность проведенного тестирования. Для этого сравним две контрольные группы, если они окажутся равны, можно быть уверенными в точности проведенного тестирования. <a id="сравнение_контрольных_групп"></a>

In [84]:
# считаем конверсию в каждое из событий от общего количества пользователей в группе: 

events_by_group['p_a1'] = (events_by_group['count_a1'] / total_users_a1).round(4)
events_by_group['p_a2'] = (events_by_group['count_a2'] / total_users_a2).round(4)

# считаем конверсию в каждое из событий от общего количества пользователей в комбинированном датасете:
events_by_group['p_combined_a1_a2'] = ((events_by_group['count_a1'] + events_by_group['count_a2']) / (total_users_a1 + total_users_a2)).round(4)

# считаем разницу конверсий:
events_by_group['difference_a1_a2'] = events_by_group['p_a1'] - events_by_group['p_a2']
events_by_group[['event', 'count_a1', 'p_a1', 'count_a2', 'p_a2', 'p_combined_a1_a2', 'difference_a1_a2']]

,event,count_a1,p_a1,count_a2,p_a2,p_combined_a1_a2,difference_a1_a2
0,main,2450,0.9867,2476,0.9857,0.9862,0.0010
1,offer,1542,0.6210,1520,0.6051,0.6130,0.0159
2,cart,1266,0.5099,1238,0.4928,0.5013,0.0171
3,payment,1200,0.4833,1158,0.4610,0.4721,0.0223


Выдвигаем гипотезы:

    Н0 - конверсии статистически равны;
    
    Н1 - конверсии статистически неравны.
    
Задаем критический уровень статистической значимости. Уровень значимости корректируется в соответствии с полным количеством статистических тестов, которые проводятся в исследовании (в каждом из тестов есть вероятность допустить ошибку первого рода). В нашем случае сравниваются 4 пары групп, 4 сравнения в каждой паре. Общее число тестов 4 * 4 = 16. Скорректируем критический уровень значимости по методу Бонферрони:

In [85]:
#alpha = (0.1/16)
alpha = (0.05/16)
alpha

0.003125

In [86]:
# считаем статистику в стандартных отклонениях стандартного нормального распределения для конверсии пользователей, перешедших на 
# главную страницу к общему количеству пользователей:
z_value_aa_0 = (events_by_group['difference_a1_a2'][0] / mth.sqrt(events_by_group['p_combined_a1_a2'][0] * (1 - events_by_group['p_combined_a1_a2'][0]) * (1/total_users_a1 + 1/total_users_a2)))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1) 

p_value_aa_0 = (1 - distr.cdf(abs(z_value_aa_0))) * 2

print('p-значение: ', p_value_aa_0)

if (p_value_aa_0 < alpha):
    print("Отвергаем нулевую гипотезу: между конверсиями есть значимая разница")
else:
    print("Принимаем гипотезу о том, что конверсии пользователей, перешедших на главную страницу, к общему количеству пользователей в контрольных группах, статистически равны")

p-значение:  0.7619612261515489
Принимаем гипотезу о том, что конверсии пользователей, перешедших на главную страницу, к общему количеству пользователей в контрольных группах, статистически равны


In [87]:
# считаем статистику в стандартных отклонениях стандартного нормального распределения для конверсии пользователей, перешедших на 
# страницу заказа товаров к общему количеству пользователей:
z_value_aa_1 = (events_by_group['difference_a1_a2'][1] / mth.sqrt(events_by_group['p_combined_a1_a2'][1] * (1 - events_by_group['p_combined_a1_a2'][1]) * (1/total_users_a1 + 1/total_users_a2)))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1) 

p_value_aa_1 = (1 - distr.cdf(abs(z_value_aa_1))) * 2

print('p-значение: ', p_value_aa_1)

if (p_value_aa_1 < alpha):
    print("Отвергаем нулевую гипотезу: между конверсиями есть значимая разница")
else:
    print("Принимаем гипотезу о том, что конверсии пользователей, перешедших на страницу заказа товаров, к общему количеству пользователей в контрольных группах, статистически равны")

p-значение:  0.24867881079343768
Принимаем гипотезу о том, что конверсии пользователей, перешедших на страницу заказа товаров, к общему количеству пользователей в контрольных группах, статистически равны


In [88]:
# считаем статистику в стандартных отклонениях стандартного нормального распределения для конверсии пользователей, перешедших
# на страницу корзины с товарами, к общему количеству пользователей:
z_value_aa_2 = (events_by_group['difference_a1_a2'][2] / mth.sqrt(events_by_group['p_combined_a1_a2'][2] * (1 - events_by_group['p_combined_a1_a2'][2]) * (1/total_users_a1 + 1/total_users_a2)))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1) 

p_value_aa_2 = (1 - distr.cdf(abs(z_value_aa_2))) * 2

print('p-значение: ', p_value_aa_2)

if (p_value_aa_2 < alpha):
    print("Отвергаем нулевую гипотезу: между конверсиями есть значимая разница")
else:
    print("Принимаем гипотезу о том, что конверсии пользователей, перешедших на страницу корзины с товарами, к общему количеству пользователей в контрольных группах, статистически равны")

p-значение:  0.2268428544087946
Принимаем гипотезу о том, что конверсии пользователей, перешедших на страницу корзины с товарами, к общему количеству пользователей в контрольных группах, статистически равны


In [89]:
# считаем статистику в стандартных отклонениях стандартного нормального распределения для конверсии пользователей, оплативших
# заказ, к общему количеству пользователей
z_value_aa_3 = (events_by_group['difference_a1_a2'][3] / mth.sqrt(events_by_group['p_combined_a1_a2'][3] * (1 - events_by_group['p_combined_a1_a2'][3]) * (1/total_users_a1 + 1/total_users_a2)))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1) 

p_value_aa_3 = (1 - distr.cdf(abs(z_value_aa_3))) * 2

print('p-значение: ', p_value_aa_3)

if (p_value_aa_3 < alpha):
    print("Отвергаем нулевую гипотезу: между конверсиями есть значимая разница")
else:
    print("Принимаем гипотезу о том, что конверсии пользователей, оплативших заказ, к общему количеству пользователей в контрольных группах, статистически равны")

p-значение:  0.114452559749056
Принимаем гипотезу о том, что конверсии пользователей, оплативших заказ, к общему количеству пользователей в контрольных группах, статистически равны


По итогам сравнения конверсий в каждое из событий в контрольных группах, можно сделать вывод, что тестирование было проведено точно и можно приступать к анализу изменений.

##### Сравним экспериментальную группу 248 с контрольной группой 246: <a id="сравнение_контрольной_группы_246_с_экспериментальной"></a>

In [90]:
# считаем конверсию в каждое из событий от общего количества пользователей в экспериментальной группе:
events_by_group['p_b'] = (events_by_group['count_b'] / total_users_b).round(4)

# считаем конверсию в каждое из событий от общего количества пользователей в комбинированном датасете:
events_by_group['p_combined_a1_b'] = ((events_by_group['count_a1'] + events_by_group['count_b']) / (total_users_a1 + total_users_b)).round(4)

# считаем разницу конверсий:
events_by_group['difference_a1_b'] = events_by_group['p_b'] - events_by_group['p_a1']
events_by_group[['event', 'count_a1', 'p_a1', 'count_b', 'p_b', 'p_combined_a1_b', 'difference_a1_b']]

,event,count_a1,p_a1,count_b,p_b,p_combined_a1_b,difference_a1_b
0,main,2450,0.9867,2493,0.9834,0.9851,-0.0033
1,offer,1542,0.6210,1531,0.6039,0.6124,-0.0171
2,cart,1266,0.5099,1230,0.4852,0.4974,-0.0247
3,payment,1200,0.4833,1181,0.4659,0.4745,-0.0174


In [91]:
# считаем статистику в стандартных отклонениях стандартного нормального распределения для конверсии пользователей, оплативших
# заказ, к общему количеству пользователей
z_value_a1b_0 = (events_by_group['difference_a1_b'][0] / mth.sqrt(events_by_group['p_combined_a1_b'][0] * (1 - events_by_group['p_combined_a1_b'][0]) * (1/total_users_a1 + 1/total_users_b)))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1) 

p_value_a1b_0 = (1 - distr.cdf(abs(z_value_a1b_0))) * 2

print('p-значение: ', p_value_a1b_0)

if (p_value_a1b_0 < alpha):
    print("Отвергаем нулевую гипотезу: между конверсиями есть значимая разница")
else:
    print("Принимаем гипотезу о том, что конверсии пользователей, перешедших на главную страницу, к общему количеству пользователей в сравниваемых группах, статистически равны")

p-значение:  0.3346946715355803
Принимаем гипотезу о том, что конверсии пользователей, перешедших на главную страницу, к общему количеству пользователей в сравниваемых группах, статистически равны


In [92]:
# считаем статистику в стандартных отклонениях стандартного нормального распределения для конверсии пользователей, перешедших на 
# страницу заказа товаров к общему количеству пользователей:
z_value_a1b_1 = (events_by_group['difference_a1_b'][1] / mth.sqrt(events_by_group['p_combined_a1_b'][1] * (1 - events_by_group['p_combined_a1_b'][1]) * (1/total_users_a1 + 1/total_users_b)))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1) 

p_value_a1b_1 = (1 - distr.cdf(abs(z_value_a1b_1))) * 2

print('p-значение: ', p_value_a1b_1)

if (p_value_a1b_1 < alpha):
    print("Отвергаем нулевую гипотезу: между конверсиями есть значимая разница")
else:
    print("Принимаем гипотезу о том, что конверсии пользователей, перешедших на страницу заказа товаров, к общему количеству пользователей в сравниваемых группах, статистически равны")

p-значение:  0.21383880931815247
Принимаем гипотезу о том, что конверсии пользователей, перешедших на страницу заказа товаров, к общему количеству пользователей в сравниваемых группах, статистически равны


In [93]:
# считаем статистику в стандартных отклонениях стандартного нормального распределения для конверсии пользователей, перешедших
# на страницу корзины с товарами, к общему количеству пользователей:
z_value_a1b_2 = (events_by_group['difference_a1_b'][2] / mth.sqrt(events_by_group['p_combined_a1_b'][2] * (1 - events_by_group['p_combined_a1_b'][2]) * (1/total_users_a1 + 1/total_users_b)))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1) 

p_value_a1b_2 = (1 - distr.cdf(abs(z_value_a1b_2))) * 2

print('p-значение: ', p_value_a1b_2)

if (p_value_a1b_2 < alpha):
    print("Отвергаем нулевую гипотезу: между конверсиями есть значимая разница")
else:
    print("Принимаем гипотезу о том, что конверсии пользователей, перешедших на страницу корзины с товарами, к общему количеству пользователей в сравниваемых группах, статистически равны")

p-значение:  0.0801831416075125
Принимаем гипотезу о том, что конверсии пользователей, перешедших на страницу корзины с товарами, к общему количеству пользователей в сравниваемых группах, статистически равны


In [94]:
# считаем статистику в стандартных отклонениях стандартного нормального распределения для конверсии пользователей, оплативших
# заказ, к общему количеству пользователей
z_value_a1b_3 = (events_by_group['difference_a1_b'][3] / mth.sqrt(events_by_group['p_combined_a1_b'][3] * (1 - events_by_group['p_combined_a1_b'][3]) * (1/total_users_a1 + 1/total_users_b)))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1) 

p_value_a1b_3 = (1 - distr.cdf(abs(z_value_a1b_3))) * 2

print('p-значение: ', p_value_a1b_3)

if (p_value_a1b_3 < alpha):
    print("Отвергаем нулевую гипотезу: между конверсиями есть значимая разница")
else:
    print("Принимаем гипотезу о том, что конверсии пользователей, оплативших заказ, к общему количеству пользователей в сравниваемых группах, статистически равны")

p-значение:  0.21715882471122328
Принимаем гипотезу о том, что конверсии пользователей, оплативших заказ, к общему количеству пользователей в сравниваемых группах, статистически равны


По итогам сравнения конверсий в каждое из событий к общему количеству пользователей контрольной группы 246 и экспериментальной группы 248 можно сделать вывод, что конверсия не изменилась и изменение шрифта не повлияло на метрики.

##### Сравним экспериментальную группу 248 с контрольной группой 247: <a id="сравнение_контрольной_группы_247_с_экспериментальной"></a>

In [95]:
# считаем конверсию в каждое из событий от общего количества пользователей в комбинированном датасете:
events_by_group['p_combined_a2_b'] = ((events_by_group['count_a2'] + events_by_group['count_b']) / (total_users_a2 + total_users_b)).round(4)

# считаем разницу конверсий:
events_by_group['difference_a2_b'] = events_by_group['p_b'] - events_by_group['p_a2']
events_by_group[['event', 'count_a2', 'p_a2', 'count_b', 'p_b', 'p_combined_a2_b', 'difference_a2_b']]

,event,count_a2,p_a2,count_b,p_b,p_combined_a2_b,difference_a2_b
0,main,2476,0.9857,2493,0.9834,0.9845,-0.0023
1,offer,1520,0.6051,1531,0.6039,0.6045,-0.0012
2,cart,1238,0.4928,1230,0.4852,0.4890,-0.0076
3,payment,1158,0.4610,1181,0.4659,0.4634,0.0049


In [96]:
# считаем статистику в стандартных отклонениях стандартного нормального распределения для конверсии пользователей, оплативших
# заказ, к общему количеству пользователей
z_value_a2b_0 = (events_by_group['difference_a2_b'][0] / mth.sqrt(events_by_group['p_combined_a2_b'][0] * (1 - events_by_group['p_combined_a2_b'][0]) * (1/total_users_a2 + 1/total_users_b)))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1) 

p_value_a2b_0 = (1 - distr.cdf(abs(z_value_a2b_0))) * 2

print('p-значение: ', p_value_a2b_0)

if (p_value_a2b_0 < alpha):
    print("Отвергаем нулевую гипотезу: между конверсиями есть значимая разница")
else:
    print("Принимаем гипотезу о том, что конверсии пользователей, перешедших на главную страницу, к общему количеству пользователей в сравниваемых группах, статистически равны")

p-значение:  0.5083830835478069
Принимаем гипотезу о том, что конверсии пользователей, перешедших на главную страницу, к общему количеству пользователей в сравниваемых группах, статистически равны


In [97]:
# считаем статистику в стандартных отклонениях стандартного нормального распределения для конверсии пользователей, перешедших на 
# страницу заказа товаров к общему количеству пользователей:
z_value_a2b_1 = (events_by_group['difference_a2_b'][1] / mth.sqrt(events_by_group['p_combined_a2_b'][1] * (1 - events_by_group['p_combined_a2_b'][1]) * (1/total_users_a2 + 1/total_users_b)))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1) 

p_value_a2b_1 = (1 - distr.cdf(abs(z_value_a2b_1))) * 2

print('p-значение: ', p_value_a2b_1)

if (p_value_a2b_1 < alpha):
    print("Отвергаем нулевую гипотезу: между конверсиями есть значимая разница")
else:
    print("Принимаем гипотезу о том, что конверсии пользователей, перешедших на страницу заказа товаров, к общему количеству пользователей в сравниваемых группах, статистически равны")

p-значение:  0.930532404341007
Принимаем гипотезу о том, что конверсии пользователей, перешедших на страницу заказа товаров, к общему количеству пользователей в сравниваемых группах, статистически равны


In [98]:
# считаем статистику в стандартных отклонениях стандартного нормального распределения для конверсии пользователей, перешедших
# на страницу корзины с товарами, к общему количеству пользователей:
z_value_a2b_2 = (events_by_group['difference_a2_b'][2] / mth.sqrt(events_by_group['p_combined_a2_b'][2] * (1 - events_by_group['p_combined_a2_b'][2]) * (1/total_users_a2 + 1/total_users_b)))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1) 

p_value_a2b_2 = (1 - distr.cdf(abs(z_value_a2b_2))) * 2

print('p-значение: ', p_value_a2b_2)

if (p_value_a2b_2 < alpha):
    print("Отвергаем нулевую гипотезу: между конверсиями есть значимая разница")
else:
    print("Принимаем гипотезу о том, что конверсии пользователей, перешедших на страницу корзины с товарами, к общему количеству пользователей в сравниваемых группах, статистически равны")

p-значение:  0.5891652187136427
Принимаем гипотезу о том, что конверсии пользователей, перешедших на страницу корзины с товарами, к общему количеству пользователей в сравниваемых группах, статистически равны


In [99]:
# считаем статистику в стандартных отклонениях стандартного нормального распределения для конверсии пользователей, оплативших
# заказ, к общему количеству пользователей
z_value_a2b_3 = (events_by_group['difference_a2_b'][3] / mth.sqrt(events_by_group['p_combined_a2_b'][3] * (1 - events_by_group['p_combined_a2_b'][3]) * (1/total_users_a2 + 1/total_users_b)))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1) 

p_value_a2b_3 = (1 - distr.cdf(abs(z_value_a2b_3))) * 2

print('p-значение: ', p_value_a2b_3)

if (p_value_a2b_3 < alpha):
    print("Отвергаем нулевую гипотезу: между конверсиями есть значимая разница")
else:
    print("Принимаем гипотезу о том, что конверсии пользователей, оплативших заказ, к общему количеству пользователей в сравниваемых группах, статистически равны")

p-значение:  0.7270594742432679
Принимаем гипотезу о том, что конверсии пользователей, оплативших заказ, к общему количеству пользователей в сравниваемых группах, статистически равны


По итогам сравнения конверсий в каждое из событий к общему количеству пользователей контрольной группы 247 и экспериментальной группы 248 можно сделать вывод, что конверсия не изменилась и изменение шрифта не повлияло на метрики.

##### Сравним экспериментальную группу 248 с объединенной контрольной группой: <a id="сравнение_объединенной_контрольной_группы_с_экспериментальной"></a>

In [100]:
# количество уникальных пользователей в объединенной контрольной группе:
total_users_a = total_users_a1 + total_users_a2

In [101]:
# считаем конверсию в каждое из событий от общего количества пользователей в объединенной группе: 

events_by_group['p_a'] = (events_by_group['count_a'] / total_users_a).round(4)

# считаем конверсию в каждое из событий от общего количества пользователей в комбинированном датасете:
events_by_group['p_combined_a_b'] = ((events_by_group['count_a'] + events_by_group['count_b']) / (total_users_a + total_users_b)).round(4)

# считаем разницу конверсий:
events_by_group['difference_a_b'] = events_by_group['p_b'] - events_by_group['p_a']
events_by_group[['event', 'count_a', 'p_a', 'count_b', 'p_b', 'p_combined_a_b', 'difference_a_b']]

,event,count_a,p_a,count_b,p_b,p_combined_a_b,difference_a_b
0,main,4926,0.9862,2493,0.9834,0.9853,-0.0028
1,offer,3062,0.6130,1531,0.6039,0.6100,-0.0091
2,cart,2504,0.5013,1230,0.4852,0.4959,-0.0161
3,payment,2358,0.4721,1181,0.4659,0.4700,-0.0062


In [102]:
# считаем статистику в стандартных отклонениях стандартного нормального распределения для конверсии пользователей, оплативших
# заказ, к общему количеству пользователей
z_value_ab_0 = (events_by_group['difference_a_b'][0] / mth.sqrt(events_by_group['p_combined_a_b'][0] * (1 - events_by_group['p_combined_a_b'][0]) * (1/total_users_a + 1/total_users_b)))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1) 

p_value_ab_0 = (1 - distr.cdf(abs(z_value_ab_0))) * 2

print('p-значение: ', p_value_ab_0)

if (p_value_ab_0 < alpha):
    print("Отвергаем нулевую гипотезу: между конверсиями есть значимая разница")
else:
    print("Принимаем гипотезу о том, что конверсии пользователей, перешедших на главную страницу, к общему количеству пользователей в сравниваемых группах, статистически равны")

p-значение:  0.34005474576086137
Принимаем гипотезу о том, что конверсии пользователей, перешедших на главную страницу, к общему количеству пользователей в сравниваемых группах, статистически равны


In [103]:
# считаем статистику в стандартных отклонениях стандартного нормального распределения для конверсии пользователей, перешедших на 
# страницу заказа товаров к общему количеству пользователей:
z_value_ab_1 = (events_by_group['difference_a_b'][1] / mth.sqrt(events_by_group['p_combined_a_b'][1] * (1 - events_by_group['p_combined_a_b'][1]) * (1/total_users_a + 1/total_users_b)))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1) 

p_value_ab_1 = (1 - distr.cdf(abs(z_value_ab_1))) * 2

print('p-значение: ', p_value_ab_1)

if (p_value_ab_1 < alpha):
    print("Отвергаем нулевую гипотезу: между конверсиями есть значимая разница")
else:
    print("Принимаем гипотезу о том, что конверсии пользователей, перешедших на страницу заказа товаров, к общему количеству пользователей в сравниваемых группах, статистически равны")

p-значение:  0.44422751462869314
Принимаем гипотезу о том, что конверсии пользователей, перешедших на страницу заказа товаров, к общему количеству пользователей в сравниваемых группах, статистически равны


In [104]:
# считаем статистику в стандартных отклонениях стандартного нормального распределения для конверсии пользователей, перешедших
# на страницу корзины с товарами, к общему количеству пользователей:
z_value_ab_2 = (events_by_group['difference_a_b'][2] / mth.sqrt(events_by_group['p_combined_a_b'][2] * (1 - events_by_group['p_combined_a_b'][2]) * (1/total_users_a + 1/total_users_b)))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1) 

p_value_ab_2 = (1 - distr.cdf(abs(z_value_ab_2))) * 2

print('p-значение: ', p_value_ab_2)

if (p_value_ab_2 < alpha):
    print("Отвергаем нулевую гипотезу: между конверсиями есть значимая разница")
else:
    print("Принимаем гипотезу о том, что конверсии пользователей, перешедших на страницу корзины с товарами, к общему количеству пользователей в сравниваемых группах, статистически равны")

p-значение:  0.18667695445518495
Принимаем гипотезу о том, что конверсии пользователей, перешедших на страницу корзины с товарами, к общему количеству пользователей в сравниваемых группах, статистически равны


In [105]:
# считаем статистику в стандартных отклонениях стандартного нормального распределения для конверсии пользователей, оплативших
# заказ, к общему количеству пользователей
z_value_ab_3 = (events_by_group['difference_a_b'][3] / mth.sqrt(events_by_group['p_combined_a_b'][3] * (1 - events_by_group['p_combined_a_b'][3]) * (1/total_users_a + 1/total_users_b)))

# задаем стандартное нормальное распределение (среднее 0, ст.отклонение 1)
distr = st.norm(0, 1) 

p_value_ab_3 = (1 - distr.cdf(abs(z_value_ab_3))) * 2

print('p-значение: ', p_value_ab_3)

if (p_value_ab_3 < alpha):
    print("Отвергаем нулевую гипотезу: между конверсиями есть значимая разница")
else:
    print("Принимаем гипотезу о том, что конверсии пользователей, оплативших заказ, к общему количеству пользователей в сравниваемых группах, статистически равны")

p-значение:  0.6104676635208746
Принимаем гипотезу о том, что конверсии пользователей, оплативших заказ, к общему количеству пользователей в сравниваемых группах, статистически равны


По итогам сравнения конверсий в каждое из событий к общему количеству пользователей объединенной контрольной группы и экспериментальной группы 248 можно сделать вывод, что конверсия не изменилась и изменение шрифта не повлияло на метрики.

### <font color=olive>Вывод</font> <a id="вывод_эксперимент"></a>

После проведения А/А/В-теста мы сравнили конверсии в каждое из событий к общему количеству пользователей попарно для следующих групп:
 - контрольные группы 246 и 247 между собой. Сравнение двух контрольных групп показало, что инструмент деления пользователей сработал верно, стат-критерии не нашли разницу между группами и дальнейшее сравнение контрольных групп с экспериментальной возможно;
 - контрольная группа 246 и экспериментальная 248. Сравнение этих групп показало, что при критическом уровне значимости, выбранном до проведения тестов, равном 0,05 и скорректированном на количество проводимых тестов, контрольная группа 246 не отличается от группы 248.
 - аналогичные результаты дали сравнения между собой контрольной группы 247 и экспериментальной 248, а также объединенной контрольной группы 246 с 247 с экспериментальной 248.
 
При изменении критического уровня значимости с аналогичной корректировкой на количество проведимых тестов до 0,1 результата проверки гипотез не меняются, критерий p-value каждой проверки для каждого события все равно выше уровня значимости даже без корректировки.

Отсюда можно сделать вывод, что различий в конверсии после изменения шрифта во всем приложении не обнаружено, а приложение не ухудшилось.

### <font color=olive> Общий вывод</font> <a id="вывод"></a>

Для анализа были представлены данные о 244 125 событиях в логах мобильного приложения по продаже продуктов питания за период с 25 июля по 07 августа 2019 года. В первоначальной таблице были изменены названия происходящих событий и названия столбцов на более удобные, а так же проведена обработка даты к стандартному виду.

При дальнейшем рассмотрении было обнаружено, что в период с 25 по 31 июля лог содержит очень мало информации за каждую из дат, а с 01 августа, судя по содержаню событий, лог можно считать полным. Для последующего анализа события за период с 24 по 31 июля были исключены.

Наиболее часто происходящее событие в приложении - это переход на главную страницу, то есть вход в него, этот тип события составлет практически полоивину всех действий за исследумый период, а именно 49%. Переход на страницу заказа товаров составляет 19% от всего количества событий, немного меньше - 18% - составляет событие добавление товара в корзину. Оплату товара произвели в 14% случаев. Так же в таблице содержится информация о 1 005 случаях прохождения туториала, но это количество настолько маленькое, что составляет менее 1% всех событий.

In [106]:
fig_three.show()

Из графиков видно, что воронки продаж без учета последовательности событий и с ней отличаются. В воронке без учета последовательности событий на шаг просмотра корзины переходит значительно больше пользователей, чем в воронке с учетом. Аналогично происходит и с шагом успешной оплаты заказа. Скорее всего это происходит потому, что у пользователей путь от прихода в приложение до покупки дольше, чем тот период, который мы выбрали. Итоговая конверсия мобильного приложения с учетом последовательности событий составила 18,33%.

После проведения А/А/В-теста мы сравнили конверсии в каждое из событий к общему количеству пользователей попарно для всех исследуемых групп. Для этого был проведен z-тест, по результатам которого можно сделать вывод, что различий в конверсии после изменения шрифта во всем приложении не обнаружено, а приложение не ухудшилось
